### Load Python Libraries

In [ ]:
!pip install pvlib
import pvlib as pv
import pandas as pd
import numpy as np
from pvlib.location import Location
from pvlib.modelchain import ModelChain
import pytz

from google.colab import drive
drive.mount('/content/drive')

### Obtaining the solar energy generation estimates

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Research/PRsolar/newdataorganized/location_coordinates_allmunicipalties.xlsx')

panel = ['crystSi','CIS','CdTe'] # different panel types
tilt = [15, 25, 35, 45]          # different tilt values
loss = [1, 5, 10, 15]            # different loss values

for tech in panel:

  for t in tilt:

    for l in loss:

      tech_sheet = pd.DataFrame({'Day' : list(range(1,5500))})

      for index, row in df.iterrows():
        lat = row['lat']
        lon = row['lon']

        # peak power is based on 1000 w/m^2 w/ a 1.6m^2 panel
        PV_result = pv.iotools.get_pvgis_hourly(lat, lon, start=2005, end=2020, raddatabase='PVGIS-ERA5', surface_tilt=t, outputformat='csv',
                                                pvcalculation=True, peakpower=1.6, pvtechchoice=tech, loss=l,
                                                url='https://re.jrc.ec.europa.eu/api/v5_2/', map_variables=True, timeout=30)
        alldata = PV_result[0]
        daily = alldata['P'].groupby(np.arange(len(alldata))//24).sum()

        tech_sheet.insert(index+1,f'{lat},{lon}',daily,True)

      tech_sheet.to_excel(f'/content/drive/MyDrive/Research/PRsolar/newdataorganized/power_calcs/{tech}_{t}deg_{l}percloss_power_2005-20.xlsx', index=False)